In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 312 kB 19.2 MB/s eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |█████████████████████████

In [3]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D 
from keras.preprocessing.image import ImageDataGenerator


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-pack

In [4]:
tensorflow.__version__

'1.14.0'

In [5]:
tensorflow.keras.__version__

'2.2.4-tf'

# image data argumentation

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_eb2dae481adf4f9caa5f03bb90f8820a = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_eb2dae481adf4f9caa5f03bb90f8820a = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_eb2dae481adf4f9caa5f03bb90f8820a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zXr7sFFYQ8_Vre2-chpRweOnHKLy4AfUh_X_CS3uTB3z',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_eb2dae481adf4f9caa5f03bb90f8820a)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_eb2dae481adf4f9caa5f03bb90f8820a.get_object(Bucket='imageclassification-donotdelete-pr-vxhuwsm4hlwxdz', Key='dataset (2).zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [10]:
import os
filenames = os.listdir('/home/wsuser/work/')

# Loading our data and performing data agumentation

In [11]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train',target_size = (64,64),batch_size = 5, color_mode='grayscale', class_mode = "categorical")
x_test= test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test',target_size = (64,64),batch_size = 5, color_mode = 'grayscale',class_mode = "categorical" )

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [12]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [13]:
print(x_test.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [14]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 99, 1: 99, 2: 99, 3: 99, 4: 99, 5: 99})

# creating the model
# Adding CNN layers

In [81]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())




# Adding dense layers

In [82]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2

In [83]:
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_7 (Dense)              (None, 6)                

# compiling the model

In [84]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model


# fitting the model

In [ ]:
 classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10
119/119 [==============================] - 6s 51ms/step - loss: 1.4366 - acc: 0.4276 - val_loss: 0.9276 - val_acc: 0.6333
Epoch 2/10
119/119 [==============================] - 5s 42ms/step - loss: 0.7432 - acc: 0.6919 - val_loss: 0.6084 - val_acc: 0.7333
Epoch 3/10
119/119 [==============================] - 5s 41ms/step - loss: 0.4840 - acc: 0.7946 - val_loss: 0.6082 - val_acc: 0.8000
Epoch 4/10
 70/119 [================>.............] - ETA: 1s - loss: 0.3291 - acc: 0.8625

# saving our model

In [55]:
classifier.save('gesture.h5')

In [56]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [57]:
ls -1

Dataset/
gesture.h5
image-classification-model_new.tgz
model.tar.gz.


In [58]:
!pip install watson-machine-learning-client --upgrade

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Requirement already up-to-date: watson-machine-learning-client in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (1.0.391)


In [59]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com",
                "apikey":"sZJr7jFKKUU8s-SPLahjAjkuwzW0wk1bDdmDmjF4FeJQ"}
client = APIClient(wml_credentials)

In [60]:
client = APIClient(wml_credentials)

In [61]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [62]:
space_uid=guid_space_name(client,'imageclassification')
print("space UID = "+space_uid)

space UID = 48be4392-4067-48db-8d4e-53440e2b5463


In [63]:
client.set.default_space(space_uid)

'SUCCESS'

In [64]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029a-cc97-56da-b8e0-39c3880dbbe7  base
tensorflow_2.1-py3.6           1eb25b8

In [65]:
software_space_uid=client.software_specifications.get_uid_by_name("default_py3.7")
software_space_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [66]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"A Gesture-based Tool for Sterile Browsing of Radiology Images CNN and Open CV",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
                                            )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [67]:
model_id

'51ff4461-88e5-417e-afe4-3e4b0972121d'

In [69]:
client.repository.download(model_id,'model.tar.gm.')

Successfully saved model content to file: 'model.tar.gm.'


'/home/wsuser/work/model.tar.gm.'

In [70]:
from keras.models import load_model
from keras.preprocessing import image

In [71]:
import tensorflow as tf
from tensorflow import keras
import h5py

# Initialization

loaded_model = keras.models.load_model('gesture.h5')

In [72]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_5 = client_eb2dae481adf4f9caa5f03bb90f8820a.get_object(Bucket='imageclassification-donotdelete-pr-vxhuwsm4hlwxdz', Key='5.png')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_5, "__iter__"): streaming_body_5.__iter__ = types.MethodType( __iter__, streaming_body_5 ) 


In [73]:
img = image.load_img(streaming_body_5,target_size = (64,64) )

In [74]:
import numpy as np
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)

# predicting

In [75]:
pred=classifier.predict_classes(x)

In [76]:
pred[0]

5

In [77]:
index=['0','1','2','3','4','5']

In [78]:
a = index[pred[0]]
print(a)

5
